In [1]:
%matplotlib inline

Matplotlib is building the font cache; this may take a moment.


# Build a neural network

Neural networks comprise of layers/modules that perform operations on data. 
The [torch.nn](https://pytorch.org/docs/stable/nn.html) namespace provides all the building blocks you need to 
build your own neural network. Every module in PyTorch subclasses the [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html). 
A neural network is a module itself that consists of other modules (layers). This nested structure allows for
building and managing complex architectures easily.

In the following sections, we'll build a neural network to classify images in the FashionMNIST dataset.

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## Get a hardware device for training

We want to be able to train our model on a hardware accelerator like the GPU, 
if it is available. Let's check to see if 
[torch.cuda](https://pytorch.org/docs/stable/notes/cuda.html) is available, else we 
continue to use the CPU.

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


## Define the class

We define our neural network by subclassing `nn.Module`, and 
initialize the neural network layers in `__init__`. Every `nn.Module` subclass implements
the operations on input data in the `forward` method.

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        # super(NeuralNetwork, self).__init__()  # Initializes internal Module state. (Python 2 & 3)
        super().__init__()  # Python 3 only
        self.flatten = nn.Flatten()  # change Size([batch, dim1, dim2, ..., dimN]) into Size([batch, dim1 * dim2 * ... * dimN])
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of `NeuralNetwork`, and move it to the `device`, and print 
it's structure.



In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


To use the model, we pass it the input data. This executes the model's `forward`,
along with some [background operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866). 
Do not call `model.forward()` directly!

Calling the model on the input returns a 10-dimensional tensor with raw predicted values for each class.
We get the prediction densities by passing it through an instance of the `nn.Softmax` module.

In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X) 
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([9], device='cuda:0')


## Model layers

Let's break down the layers in the FashionMNIST model. To illustrate it, we 
will take a sample minibatch of 3 images of size **28x28** and see what happens to it as 
we pass it through the network. 



In [7]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten

We initialize the [nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html) 
layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values (the minibatch dimension (at dim=0) is maintained).



In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear 

The [linear layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)
is a module that applies a linear transformation on the input using it's stored weights and biases.

In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU

Non-linear activations are what create the complex mappings between the model's inputs and outputs.
They are applied after linear transformations to introduce *nonlinearity*, helping neural networks
learn a wide variety of phenomena.

In this model, we use [nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html) between our
linear layers, but there's other activations to introduce non-linearity in your model.

In [10]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.7047, -0.3237,  0.4558, -0.2242, -0.1404, -0.0499, -0.0537,  0.1231,
          0.1527,  0.3479, -0.3934, -0.0551, -0.4923, -0.2033, -0.3598, -0.2526,
         -0.0144, -0.4399,  0.1854, -0.4894],
        [ 0.6809, -0.2726,  0.4622, -0.2576, -0.3245, -0.0986, -0.0718, -0.0822,
          0.3755,  0.2411, -0.4129, -0.0315, -0.6605,  0.1638, -0.3383, -0.3349,
          0.1347, -0.2810,  0.0266, -0.3108],
        [ 0.4367, -0.3616,  0.4141, -0.2697, -0.2321,  0.1637, -0.0419,  0.3736,
          0.2428,  0.0807, -0.1117, -0.1469, -0.3766,  0.0805, -0.1058, -0.0313,
         -0.1413, -0.2917,  0.1283, -0.5067]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.7047, 0.0000, 0.4558, 0.0000, 0.0000, 0.0000, 0.0000, 0.1231, 0.1527,
         0.3479, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.1854, 0.0000],
        [0.6809, 0.0000, 0.4622, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3755,
         0.2411, 0.0000, 0.0000, 0.0000, 0.1638, 0.000

### nn.Sequential

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) is an ordered 
container of modules. The data is passed through all the modules in the same order as defined. You can use
sequential containers to put together a quick network like `seq_modules`.



In [11]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
print(logits)

tensor([[-0.0817, -0.0208, -0.0728, -0.1524,  0.0177, -0.2535,  0.3069, -0.1353,
          0.2961,  0.1679],
        [-0.1744, -0.0656, -0.0943, -0.0654, -0.0006, -0.1584,  0.2370, -0.1250,
          0.2865,  0.1173],
        [-0.1620, -0.0329, -0.1356, -0.0883,  0.0007, -0.2170,  0.4157, -0.2847,
          0.4192,  0.0725]], grad_fn=<AddmmBackward>)


### nn.Softmax

The last linear layer of the neural network returns `logits` - raw values in \[`-infty`, `infty`] - which are passed to the
[nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html) module. The logits are scaled to values \[0, 1\] representing the model's predicted densities for each class. `dim` parameter indicates the dimension along which the values must sum to 1.

In [12]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab)

tensor([[0.0900, 0.0956, 0.0908, 0.0838, 0.0994, 0.0758, 0.1327, 0.0853, 0.1313,
         0.1155],
        [0.0833, 0.0929, 0.0903, 0.0929, 0.0991, 0.0847, 0.1257, 0.0875, 0.1321,
         0.1115],
        [0.0828, 0.0942, 0.0850, 0.0891, 0.0974, 0.0783, 0.1475, 0.0732, 0.1480,
         0.1046]], grad_fn=<SoftmaxBackward>)


## Model parameters

Many layers inside a neural network are *parameterized*, i.e. have associated weights 
and biases that are optimized during training. Subclassing `nn.Module` automatically 
tracks all fields defined inside your model object, and makes all parameters 
accessible using your model's `parameters()` or `named_parameters()` methods.

In this example, we iterate over each parameter, and print its size and a preview of its values.


In [13]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")  # only print part of the params

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0151,  0.0334, -0.0342,  ..., -0.0219,  0.0073,  0.0117],
        [-0.0293, -0.0206,  0.0153,  ...,  0.0286, -0.0075, -0.0023]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0221, 0.0178], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0094, -0.0294,  0.0063,  ..., -0.0051,  0.0360, -0.0379],
        [-0.0165, -0.0104,  0.0378,  ..., -0.0390,  0.0351,  0.0061]],
       device='cuda:0

--------------


